In [ ]:
import SimpleITK as sitk
import numpy as np
from skimage.segmentation import slic

# normalization (masked region)
def min_max_normalization(image_array, mask_array):
    masked_data = image_array[mask_array > 0]
    min_val = np.min(masked_data)
    max_val = np.max(masked_data)
    normalized_array = (image_array - min_val) / (max_val - min_val)
    return normalized_array

# determining the number of supervoxels according to tumor volume
def determine_superpixels(mask_array):
    mask_sum = np.sum(mask_array> 0) 
    if mask_sum < 1000:
        return 30
    elif 1000 <= mask_sum < 10000:
        return 50
    elif 10000 <= mask_sum < 100000:
        return 60
    else:
        return 80

# SLIC
def process_mri_and_entropy(mri_image_path, entropy_image_path, tumor_mask_path, output_dir):
        
        # Load MRI, entropy, and tumor mask images
        mri_image = sitk.ReadImage(mri_image_path)
        entropy_image = sitk.ReadImage(entropy_image_path)
        tumor_mask = sitk.ReadImage(tumor_mask_path)

        # Convert images to numpy arrays and downcast to float32
        mri_array = sitk.GetArrayFromImage(mri_image).astype(np.float32)
        entropy_array = sitk.GetArrayFromImage(entropy_image).astype(np.float32)
        tumor_mask_array = sitk.GetArrayFromImage(tumor_mask).astype(np.float32)

        # normalization (masked region)
        mri_array_normalized = min_max_normalization(mri_array, tumor_mask_array)
        entropy_array_normalized = min_max_normalization(entropy_array, tumor_mask_array)

        # number of supervoxels
        num_superpixels = determine_superpixels(tumor_mask_array)

        # Combine MRI and entropy values into a multi-channel image
        multi_channel_image = np.stack([mri_array_normalized, entropy_array_normalized], axis=-1)

        # Apply 3D SLIC segmentation to the masked region
        segmented_tumor_region = slic(multi_channel_image, n_segments=num_superpixels, compactness=0.05, mask=tumor_mask_array)

        # save
        segmented_tumor_image = sitk.GetImageFromArray(segmented_tumor_region)
        segmented_tumor_image.CopyInformation(mri_image)
        
        # Save the segmented tumor region
        sitk.WriteImage(segmented_tumor_image, output_dir)

# example usage
mri_image_dir = 'File path'
entropy_image_dir = 'File path'
tumor_mask_dir = 'File path'
output_dir = 'File path'

process_mri_and_entropy(mri_image_dir, entropy_image_dir, tumor_mask_dir, output_dir)
